#Midterm
Steps

1) Data pre-processing:
Clean and prepare your data by removing irrelevant characters, handling missing values, and converting text to numerical features using NLP techniques.

2) Feature engineering:
Extract relevant features from the reviews like sentiment scores, keyword counts, and review length.

3) Model selection and tuning:
Try different models and tune hyperparameters to find the best performing one for your data.

4) Evaluation metrics:
Use metrics like Mean Squared Error (MSE), Mean Absolute Error (MAE), or R-squared to assess the model's performance.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#imports
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import nltk
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb

In [ ]:
#loading the data set
train_data = pd.read_csv("/content/drive/MyDrive/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/test.csv")
train_data.head()

In [ ]:
# Merge test_data with train_data to get metadata for each 'Id'
test_data = test_data.merge(train_data, on="Id", how="left")
test_data.head() #will deal with processing later....

#Step 1: Data Preprocessing/Cleaning

In [ ]:
#Fixing Helpfulness, no Nans
train_data.loc[train_data['HelpfulnessDenominator'] == 0, 'HelpfulnessDenominator'] = 1
train_data['HelpfulnessNumerator'] = train_data['HelpfulnessNumerator'].fillna(0)
train_data['HelpfulnessDenominator'] = train_data['HelpfulnessDenominator'].fillna(1)

#dropping all rows where text is missing cuz thats kinda the main thing we are basing our system off of
train_data = train_data.dropna(subset=["Text"])
train_data = train_data.dropna(subset=["Summary"])

#dropping all irrelvant rows that dont add value to the prediction.... maybe add Summary back here
train_data = train_data.drop(columns=["Id","ProductId","UserId"])
train_data = train_data.dropna(subset=["Score"])
train_data.head()

#Step 2: Feature Engineering

In [ ]:
#Creating new feature FullText that is just a combo of Summary and Text & dropping now irrelvant features
train_data['FullText'] = train_data['Summary'] + " " + train_data['Text']
train_data = train_data.drop(columns=["Summary","Text"])

#Creating new feature Helpfulness Ratio & dropping now irrelevant features
train_data['HelpfulnessRatio'] = train_data['HelpfulnessNumerator'] / (train_data['HelpfulnessDenominator'] + 1e-5)
train_data = train_data.drop(columns=["HelpfulnessNumerator","HelpfulnessDenominator"])
train_data.head()

In [ ]:
#Steps for getting rid of all the stop words
nltk.download('stopwords')
stop_words = stopwords.words('english')

def preprocess_text(text):
    #Converting to lowercase
    text = str(text).lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stop words
    return ' '.join([word for word in text.split() if word not in stop_words])

#train_data = train_data.sample(n=20000, random_state=42) #using this so I can make processing faster
train_data['FullText'] = train_data['FullText'].apply(preprocess_text)
train_data.head()

In [ ]:
# Use TF-IDF on the 'Text' column to convert text to numerical features
tfidf = TfidfVectorizer(max_df=0.05, max_features=500, ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(train_data["FullText"])

#Reducing the TFIDF matrix
n_components = 50  # Choose the number of components
svd = TruncatedSVD(n_components=n_components)
reduced_matrix = svd.fit_transform(tfidf_matrix)

#Getting our X and y
X = pd.concat([train_data[["HelpfulnessRatio", "Time"]].reset_index(drop=True),pd.DataFrame(reduced_matrix)], axis=1)
X.columns = X.columns.astype(str)
y = train_data["Score"]

In [ ]:
X.head()

#Step 3.1: Model Selection

In [ ]:
#creating training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
#Random Forest Model
rf_model = RandomForestClassifier(class_weight='balanced',random_state=42, max_depth=12, min_samples_leaf=4,min_samples_split=2,n_estimators=300)
rf_model.fit(X_train, y_train)

# Make predictions on the test set for random Forest
y_predrf = rf_model.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, y_predrf))
print("cross-validation score:")
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
print(cv_scores.mean())

In [ ]:
#XGB model
y_train = y_train.astype(int)
y_test = y_test.astype(int)
y_train = y_train - 1
y_test = y_test - 1
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

y_predXGB = xgb_model.predict(X_test)
print("XGB:")
print(classification_report(y_test, y_predXGB))
print("cross-validation score:")
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5)
print(cv_scores.mean())

#Step 3.2: Comparing Models & Tuning Hyper Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

# Example for a Random Forest model
param_grid = {
    'n_estimators': [10, 20, 70],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(),
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

#Step 4: Predicting our Testing Data & Evaluating

In [ ]:
test_data.head()

In [ ]:
#Prepare the testing data for prediction
#Fixing Helpfulness, no Nans
test_data.loc[test_data['HelpfulnessDenominator'] == 0, 'HelpfulnessDenominator'] = 1
test_data['HelpfulnessNumerator'] = test_data['HelpfulnessNumerator'].fillna(0)
test_data['HelpfulnessDenominator'] = test_data['HelpfulnessDenominator'].fillna(1)

#dropping all rows where text is missing cuz thats kinda the main thing we are basing our system off of
test_data = test_data.dropna(subset=["Text"])
test_data = test_data.dropna(subset=["Summary"])

#dropping all irrelvant rows that dont add value to the prediction.... maybe add Summary back here
test_data = test_data.drop(columns=["ProductId","UserId","Score_y"])

#Creating new feature FullText that is just a combo of Summary and Text & dropping now irrelvant features
test_data['FullText'] = test_data['Summary'] + " " + test_data['Text']
test_data = test_data.drop(columns=["Summary","Text"])

#Creating new feature Helpfulness Ratio & dropping now irrelevant features
test_data['HelpfulnessRatio'] = test_data['HelpfulnessNumerator'] / (test_data['HelpfulnessDenominator'] + 1e-5)
test_data = test_data.drop(columns=["HelpfulnessNumerator","HelpfulnessDenominator"])
test_data.head()

##test_data = test_data.sample(n=1000, random_state=42)
test_data['FullText'] = test_data['FullText'].apply(preprocess_text)

# Use TF-IDF on the 'Text' column to convert text to numerical features
tfidf = TfidfVectorizer(max_df=0.05, max_features=500, ngram_range=(1, 2))
tfidf_matrix = tfidf.fit_transform(test_data["FullText"])

#Reducing the TFIDF matrix
n_components = 50  # Choose the number of components
svd = TruncatedSVD(n_components=n_components)
reduced_matrix = svd.fit_transform(tfidf_matrix)

#Getting our X and y
X = pd.concat([test_data[["HelpfulnessRatio", "Time"]].reset_index(drop=True),pd.DataFrame(reduced_matrix)], axis=1)
X.columns = X.columns.astype(str)

In [ ]:
test_data.head()

In [ ]:
test_data["Score_x"] = rf_model.predict(X)

In [ ]:
test_data.head()

In [ ]:
from matplotlib import pyplot as plt
test_data['Score_x'].plot(kind='hist', bins=20, title='Score_x')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#Saving Predictions for Submission
submission = test_data[["Id", "Score_x"]]
submission.to_csv("predictions.csv", index=False)